In [40]:
from database.market import Market
import pandas as pd
from statistics import variance
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer

In [37]:
market = Market()
market.connect()
tickers = market.retrieve_data("sp500")["Symbol"]
market.close()

In [82]:
test = True
if test:
    ticker = "AAPL"
    pdr_data = market.retrieve_price_data("pdr_prices",ticker)
    pdr_data = DateTransformer.convert_to_date(ColumnTransformer.rename_columns(pdr_data,"_"),"date")[["date","ticker","adjclose"]]
    quandl_data = market.retrieve_price_data("finnhub_prices",ticker)
    quandl_data["date"] = [datetime.fromtimestamp(x) for x in quandl_data["t"]]
    quandl_data.rename(columns={"c":"adjclose_finnhub"},inplace=True)
    quandl_data = DateTransformer.convert_to_date(ColumnTransformer.rename_columns(quandl_data,""),"date")[["date","ticker","adjclose_finnhub"]]
    tiingo_data = market.retrieve_price_data("tiingo_prices",ticker)
    tiingo_data = DateTransformer.convert_to_date(ColumnTransformer.rename_columns(tiingo_data,""),"date")[["date","ticker","adjclose"]]
    pdr_data.reset_index(drop=True,inplace=True)
    quandl_data.reset_index(drop=True,inplace=True)
    tiingo_data.reset_index(drop=True,inplace=True)
    pdr_data["date_string"] = [datetime.strftime(x,"%Y-%m-%d") for x in pdr_data["date"]]
    quandl_data["date_string"] = [datetime.strftime(x,"%Y-%m-%d") for x in quandl_data["date"]]
    tiingo_data["date_string"] = [datetime.strftime(x,"%Y-%m-%d") for x in tiingo_data["date"]]
    lel = pdr_data.merge(tiingo_data,suffixes=("_pdr","_tiingo"),on=["date_string","ticker"],how="left")
    final = lel.merge(quandl_data,suffixes=("_lel","_tiingo"),on=["date_string","ticker"],how="left")
    v = []
    for i in range(len(final)):
        row = final.iloc[i]
        var = variance([row["adjclose_pdr"],row["adjclose_finnhub"],row["adjclose_tiingo"]])
        v.append(var)
    final["var"] = v
    max_var = final["var"].max()
    weird = final[final["var"] == max_var]["date"].item()
    mean_var = final["var"].mean()
    instances = len(final[final["var"] > 0.005])
    record = {"ticker":ticker,"max_v":max_var,"mean_v":mean_var,"instances":instances,"extreme":weird}

In [85]:
final

,date_pdr,ticker,adjclose_pdr,date_string,date_tiingo,adjclose_tiingo,date,adjclose_finnhub,var
0,2008-01-02 00:00:00+00:00,AAPL,6.002815,2008-01-02,2008-01-02 00:00:00+00:00,6.002071,2008-01-02 19:00:00+00:00,6.961779,0.306775
1,2008-01-03 00:00:00+00:00,AAPL,6.005590,2008-01-03,2008-01-03 00:00:00+00:00,6.004843,2008-01-03 19:00:00+00:00,6.430350,0.060246
2,2008-01-04 00:00:00+00:00,AAPL,5.547152,2008-01-04,2008-01-04 00:00:00+00:00,5.546463,NaT,NaN,NaN
3,2008-01-07 00:00:00+00:00,AAPL,5.472903,2008-01-07,2008-01-07 00:00:00+00:00,5.472222,2008-01-07 19:00:00+00:00,6.116071,0.138035
4,2008-01-08 00:00:00+00:00,AAPL,5.276032,2008-01-08,2008-01-08 00:00:00+00:00,5.275378,2008-01-08 19:00:00+00:00,6.407139,0.426715
...,...,...,...,...,...,...,...,...,...
3286,2021-01-21 00:00:00+00:00,AAPL,136.869995,2021-01-21,2021-01-21 00:00:00+00:00,136.870000,2021-01-21 19:00:00+00:00,139.070000,1.613337
3287,2021-01-22 00:00:00+00:00,AAPL,139.070007,2021-01-22,2021-01-22 00:00:00+00:00,139.070000,NaT,NaN,NaN
3288,2021-01-25 00:00:00+00:00,AAPL,142.919998,2021-01-25,2021-01-25 00:00:00+00:00,142.920000,2021-01-25 19:00:00+00:00,143.160000,0.019200
3289,2021-01-26 00:00:00+00:00,AAPL,143.160004,2021-01-26,2021-01-26 00:00:00+00:00,143.160000,2021-01-26 19:00:00+00:00,142.060000,0.403335


In [32]:
quandl_data

,_id,index,adjclose,h,l,o,s,t,v,ticker,date
0,6012af47786432b430050fd5,0,6.958561,7.152139,6.876779,7.116779,ok,1199232000,38542058,AAPL,2008-01-01 19:00:00
1,6012af47786432b430050fd6,1,6.961779,7.049639,6.881779,6.978921,ok,1199318400,30073810,AAPL,2008-01-02 19:00:00
2,6012af47786432b430050fd7,2,6.430350,6.892850,6.388921,6.837489,ok,1199404800,51993955,AAPL,2008-01-03 19:00:00
3,6012af47786432b430050fd8,3,6.344279,6.557139,6.079639,6.473211,ok,1199664000,74006920,AAPL,2008-01-06 19:00:00
4,6012af47786432b430050fd9,4,6.116071,6.516421,6.099989,6.433561,ok,1199750400,54422038,AAPL,2008-01-07 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...
3284,6012af47786432b430051ca9,3284,136.870000,139.670000,133.590000,133.800000,ok,1611187200,120529544,AAPL,2021-01-20 19:00:00
3285,6012af47786432b430051caa,3285,139.070000,139.850000,135.020000,136.280000,ok,1611273600,114459360,AAPL,2021-01-21 19:00:00
3286,6012af47786432b430051cab,3286,142.920000,145.090000,136.540000,143.070000,ok,1611532800,157611713,AAPL,2021-01-24 19:00:00
3287,6012af47786432b430051cac,3287,143.160000,144.300000,141.370000,143.600000,ok,1611619200,98390555,AAPL,2021-01-25 19:00:00


In [86]:
records = []
fails = []
for ticker in tqdm(tickers):
    try:
        pdr_data = market.retrieve_price_data("pdr_prices",ticker)
        pdr_data = DateTransformer.convert_to_date(ColumnTransformer.rename_columns(pdr_data,"_"),"date")[["date","ticker","adjclose"]]
        quandl_data = market.retrieve_price_data("finnhub_prices",ticker)
        quandl_data["date"] = [datetime.fromtimestamp(x) for x in quandl_data["t"]]
        quandl_data.rename(columns={"c":"adjclose_finnhub"},inplace=True)
        quandl_data = DateTransformer.convert_to_date(ColumnTransformer.rename_columns(quandl_data,""),"date")[["date","ticker","adjclose_finnhub"]]
        tiingo_data = market.retrieve_price_data("tiingo_prices",ticker)
        tiingo_data = DateTransformer.convert_to_date(ColumnTransformer.rename_columns(tiingo_data,""),"date")[["date","ticker","adjclose"]]
        pdr_data.reset_index(drop=True,inplace=True)
        quandl_data.reset_index(drop=True,inplace=True)
        tiingo_data.reset_index(drop=True,inplace=True)
        pdr_data["date_string"] = [datetime.strftime(x,"%Y-%m-%d") for x in pdr_data["date"]]
        quandl_data["date_string"] = [datetime.strftime(x,"%Y-%m-%d") for x in quandl_data["date"]]
        tiingo_data["date_string"] = [datetime.strftime(x,"%Y-%m-%d") for x in tiingo_data["date"]]
        lel = pdr_data.merge(tiingo_data,suffixes=("_pdr","_tiingo"),on=["date_string","ticker"],how="left")
        final = lel.merge(quandl_data,suffixes=("_lel","_tiingo"),on=["date_string","ticker"],how="left")
        v = []
        for i in range(len(final)):
            row = final.iloc[i]
            var = variance([row["adjclose_pdr"],row["adjclose_finnhub"],row["adjclose_tiingo"]])
            v.append(var)
        final["var"] = v
        max_var = final["var"].max()
        weird = final[final["var"] == max_var]["date"].item()
        mean_var = final["var"].mean()
        instances = len(final[final["var"] > 0.005])
        record = {"ticker":ticker,"max_v":max_var,"mean_v":mean_var,"instances":instances,"extreme":weird}
        records.append(record)
    except Exception as e:
        print(ticker,str(e))
        fails.append({"ticker":ticker,"error":str(e)})

 18%|████████████████████████▏                                                                                                            | 92/505 [02:46<06:30,  1.06it/s]

CDW 't'
CE 't'


 19%|█████████████████████████▎                                                                                                           | 96/505 [02:48<04:05,  1.67it/s]

CNP can only convert an array of size 1 to a Python scalar


 24%|███████████████████████████████▎                                                                                                    | 120/505 [03:14<06:59,  1.09s/it]

...

WRK can only convert an array of size 1 to a Python scalar


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 501/505 [09:54<00:04,  1.07s/it]

ZBRA 't'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [09:56<00:00,  1.18s/it]

ZION 't'
ZTS 't'


In [87]:
analysis = pd.DataFrame(records)

In [94]:
analysis.sort_values("mean_v",ascending=False).head(50)

,ticker,max_v,mean_v,instances,extreme
343,NVR,71982.014873,574.192123,2557,2020-03-11 20:00:00+00:00
69,BLK,2068.600593,558.194992,2576,2008-07-16 20:00:00+00:00
71,BKNG,23056.333333,454.115137,659,2017-11-06 19:00:00+00:00
245,IBM,1137.773364,391.883712,2571,2013-01-22 19:00:00+00:00
408,SPG,906.369629,347.470618,2566,2015-01-21 19:00:00+00:00
377,PSA,770.823382,284.542272,2575,2016-03-10 19:00:00+00:00
168,ESS,1015.121229,261.668000,2576,2008-09-17 20:00:00+00:00
31,AIG,12001.299930,239.408957,2561,2008-01-22 19:00:00+00:00
153,DD,1745.153016,235.051566,642,2017-08-31 20:00:00+00:00
23,GOOG,3074.561002,231.545251,23,2021-01-19 19:00:00+00:00


In [88]:
strange = analysis[analysis["mean_v"] > 1].sort_values("mean_v",ascending=False)

In [89]:
len(strange)

425

In [ ]:
ticker = "C"
pdr_data = market.retrieve_price_data("pdr_prices",ticker)
quandl_data = market.retrieve_price_data("finnhub_prices",ticker)
tiingo_data = market.retrieve_price_data("tiingo_prices",ticker)
pdr_data.index = pd.to_datetime(pdr_data["Date"],utc=True)
quandl_data.index = pd.to_datetime(quandl_data["Date"],utc=True)
tiingo_data.index = pd.to_datetime(tiingo_data["date"],utc=True)
lel = pdr_data.join(tiingo_data,lsuffix="pdr",rsuffix="tiingo")
final = lel.join(quandl_data,rsuffix="quandl")
final["date"] = [datetime.strptime(x.split("T")[0],"%Y-%m-%d") for x in final["date"]]
final = final[final["date"]>datetime(2010,1,1)]
plt.plot(final["Adj_Close"])
plt.plot(final["Adj Close"])
plt.plot(final["adjClose"])
plt.legend(["quandl","pdr","tiingo"])
plt.title(ticker)
plt.show()

In [ ]:
for ticker in strange["ticker"]:
    pdr_data = market.retrieve_price_data("pdr_prices",ticker)
    quandl_data = market.retrieve_price_data("finnhub_prices",ticker)
    tiingo_data = market.retrieve_price_data("tiingo_prices",ticker)
    pdr_data.index = pd.to_datetime(pdr_data["Date"],utc=True)
    quandl_data.index = pd.to_datetime(quandl_data["Date"],utc=True)
    tiingo_data.index = pd.to_datetime(tiingo_data["date"],utc=True)
    lel = pdr_data.join(tiingo_data,lsuffix="pdr",rsuffix="tiingo")
    final = lel.join(quandl_data,rsuffix="quandl")
    plt.plot(final["Adj_Close"])
    plt.plot(final["Adj Close"])
    plt.plot(final["adjClose"])
    plt.legend(["quandl","pdr","tiingo"])
    plt.show()

In [ ]:
final[final["var"] > 0.001][["Adj Close","Adj_Close","adjClose"]]